# ReAct Agent + OpenAI Functions


#### 필수 라이브러리 설치

In [179]:
!pip install -qU langchain langchain-openai langchain_community langchain_experimental langchain-text-splitters tiktoken pypdf faiss-cpu

#### Openai api key 등록

In [167]:
import os

os.environ["OPENAI_API_KEY"] = 'sk-**********************************************************'

## 1.ReAct Agent



### Tools

#### Tavily Search

In [159]:
import os

os.environ["TAVILY_API_KEY"] = "tvly-***********************************************"
os.environ["LANGCHAIN_PROJECT"] = "default"

In [160]:
from langchain_community.tools.tavily_search import TavilySearchResults

#정답 5가지 설정
search = TavilySearchResults(k=5)

In [161]:
#테스트
search.invoke("양재 시민의 숲에서 가장 가까운 코스트코는 어디인가요?")

[{'url': 'https://m.blog.naver.com/sh2553/223159521355',
  'content': '사이드 메뉴를 다양하게 판매하고 있어 입이 즐겁다. 점심시간에 방문하면 웨이팅이 있을 정도로. 유명하고 맛있는 즉석 떡볶이 맛집이라. 조금 이른시간 혹은 조금 늦게 방문을 추천! 재방문 지수 : ⭐⭐⭐⭐⭐. ⬇️빨강떡볶이 후기 보러가기⬇️. https://blog ...'},
 {'url': 'https://blackbear88.tistory.com/entry/양재양재시민의숲-코스트코-근처-N번째-방문-카레-돈까스-맛집-“아르코”내돈-내산-솔직후기',
  'content': '양재/양재시민의숲 코스트코 근처 맑은 국물이 맛있는 곰탕 맛집 "남도나주곰탕",내돈 내산 솔직후기 (359) 2023.09.22 잠실/잠실새내 롯데타워 고소한 치즈를 활용한 브런치 맛집 "치즈룸&테이스팅룸",내돈 내산 솔직후기 (349)'},
 {'url': 'https://blog.naver.com/PostView.nhn?blogId=abc204308&logNo=223241678694',
  'content': '몽거는 양재시민의 숲에서. 도보로는 15분거리에 위치하고 있어! 주변에 이마트, 코스트코 등 큰 쇼핑센터가 있어서. 마을버스도 다니고 있어! 양재시민의숲역 카페에서 찾기 쉽지않은. 주차가능한 카페라서 차 가져오시는 분들은. 편하게 카페 이용이 가능할 것 ...'},
 {'url': 'https://www.costco.co.kr/store-finder/Yangjae',
  'content': '06772. 서울특별시 서초구 양재대로 159. (양재동 217-6) 전화번호: 1899-9900. 이메일: member@costcokr.com. 길 찾기.'},
 {'url': 'https://moovitapp.com/index/ko/대중_교통-코스트코_양재점-서울시Seoul-site_8477366-1802',
  'content': "코스트코 양재점 까지 가장 

In [201]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

from langchain import hub

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")


In [202]:
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

### Construct the OpenAI Functions agent

**create_openai_functions_agent**

In [203]:
from langchain.agents import create_openai_functions_agent, tool

#함수 에이전트 생성
agent_tools = [search]
agent = create_openai_functions_agent(llm, agent_tools, prompt)

In [204]:
from langchain.agents import AgentExecutor

#에이전트 실행기 생성
agent_executor = AgentExecutor(agent=agent, tools=agent_tools, verbose=True)

In [205]:
#테스트
agent_executor.invoke({"input": "hi!"})



> Entering new AgentExecutor chain...
Hello! How can I assist you today?

> Finished chain.


{'input': 'hi!', 'output': 'Hello! How can I assist you today?'}

In [207]:
#테스트
result = agent_executor.invoke(
    {"input": "랭체인의 공식 홈페이지 주소를 알려줘요"}
)

print(result["output"])



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': '랭체인 공식 홈페이지 주소'}`


[{'url': 'https://www.samsungsds.com/kr/insights/what-is-langchain.html', 'content': '인공지능\n랭체인(LangChain)이란 무엇인가?\n인쇄하기\n이 글은 IDG의 아티클을 전재하여 제공합니다.\n 랭체인은 간단하고 통합된 API를 노출하여 기본 LLM의 구현 세부 사항을 요약하는데, 이 API를 통해 개발자들은 코드를 크게 변경하지 않고 모델을 쉽게 교체하거나 대체할 수 있다.\n 구독하기\n좋아요\nInfoWorld\n뉴스레터 구독신청 ①\xa0 데이터 소스\n애플리케이션이 LLM에 대한 컨텍스트를 구축하기 위해 PDF, 웹 페이지, CSV, 관계형 데이터베이스와 같은 외부 소스에서 데이터를 검색해야 하는 경우가 있다. 오픈소스 및 상용 LLM, 벡터 데이터베이스, 데이터 소스 및 임베딩에 대한 지원은 랭체인을 개발자에게 필수적인 도구로 만들고 있다.\n'}, {'url': 'https://www.langchain.com/', 'content': 'Join Discord\nContribute on GitHub\n5.3M\nMonthly Downloads\n67k\nGithub Stars\n1800+\nContributors\n34k+\nApps Powered\nReady to build?\nGo To Docs\nGo To Docs\nGo To Docs\nContact Sales\nContact Sales\nContact Sales\nProducts and use-cases\nLangChain\nLangSmith\nRetrieval\nAgents\nInspiration\nCode\nGitHub\nLangChain Hub\nPython Docs\nJS/TS Docs\nSocial\nT

**Create tool**

In [222]:
from langchain_core.tools import tool


@tool
def multiply(first_int: int, second_int: int) -> int:
    """Multiply two integers together."""
    return first_int * second_int

@tool
def add(first_int: int, second_int: int) -> int:
    "Add two integers."
    return first_int + second_int


@tool
def exponentiate(base: int, exponent: int) -> int:
    "Exponentiate the base to the exponent power."
    return base**exponent


tools = [multiply, add, exponentiate]

**create_openai_tool_agent**

In [212]:
from langchain.agents import AgentExecutor, create_openai_tools_agent

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-tools-agent")

llm = ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=0)

tool_agent = create_openai_tools_agent(llm, tools, prompt)

In [213]:
#툴 에이전트 실행기 생성
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [214]:
#테스트
agent_executor.invoke({"input": "what is LangChain?"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'LangChain'}`


[{'url': 'https://www.langchain.com/langchain', 'content': 'Augment the power\nof\xa0LLMs with your data\nLangChain helps connect LLMs to your company’s private sources\nof data and APIs to create context-aware, reasoning applications.\n Our Methods\nReady to start shipping\nreliable GenAI apps faster?\nLangChain and LangSmith are critical parts of the reference\narchitecture to get you from prototype to production. The largest community building the future of LLM apps\nLangChain’s flexible abstractions and AI-first toolkit make it\xa0the\xa0#1\xa0choice for developers when building with GenAI.\n Why choose LangChain?\nLangChain is easy to get started with and\xa0gives\xa0you choice, flexibility, and power as\xa0you scale.\n Get customizability and control with a durable runtime baked in\nLangChain Expression Language (LCEL) lets you build your app in a truly composable way, 

{'input': 'what is LangChain?',
 'output': 'LangChain is a platform that helps connect Large Language Models (LLMs) to private sources of data and APIs to create context-aware, reasoning applications. It provides developers with flexible abstractions and an AI-first toolkit to build with GenAI. LangChain offers customizability, control, and a durable runtime, making it easy to get started and scale applications. LangChain Expression Language (LCEL) allows developers to build apps in a composable way and customize them as needed. LangChain was launched in October 2022 as an open-source project and has gained popularity for its capabilities in document analysis and summarization, chatbots, and code analysis. It has integrations with various systems and tools to support a wide range of use cases.'}

In [229]:
# Choose the LLM that will drive the agent
# Only certain models support this
model = ChatOpenAI(model="gpt-4", temperature=0)

# Construct the OpenAI Tools agent
agent = create_openai_tools_agent(model, tools, prompt)

# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [234]:
agent_executor.invoke(
    {
        "input": "Take 3 to the fifth power and multiply that by the sum of twelve and three, then square the whole result"
    }
)



> Entering new AgentExecutor chain...

Invoking: `exponentiate` with `{'base': 3, 'exponent': 5}`


243
Invoking: `add` with `{'first_int': 12, 'second_int': 3}`


15Are followup questions needed here: No.
Intermediate answer: 3 to the fifth power is 243. The sum of twelve and three is 15.
Follow up: Multiply 243 by 15.
Intermediate answer: 243 multiplied by 15 is 3645.
Follow up: Square 3645.
Intermediate answer: The square of 3645 is 13302025.
So the final answer is: 13302025

> Finished chain.


{'input': 'Take 3 to the fifth power and multiply that by the sum of twelve and three, then square the whole result',
 'output': 'Are followup questions needed here: No.\nIntermediate answer: 3 to the fifth power is 243. The sum of twelve and three is 15.\nFollow up: Multiply 243 by 15.\nIntermediate answer: 243 multiplied by 15 is 3645.\nFollow up: Square 3645.\nIntermediate answer: The square of 3645 is 13302025.\nSo the final answer is: 13302025'}